In [42]:
import requests
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from urllib.parse import quote
from bs4 import BeautifulSoup
from selenium import webdriver

In [43]:
url = 'https://www.ggpolice.go.kr/main/html.do?menu=HC44'
res = requests.get(url)
soup = BeautifulSoup(res.text,'html.parser')

In [44]:
lis = soup.select('li.road')
len(lis)

31

In [45]:
li = lis[0]
name = li.find('a').get_text().strip()
link = li.find('a')['href']
link, name

('https://www.ggpolice.go.kr/swjb/html.do?menu=HC24', '수원중부서')

In [46]:
sub_res = requests.get(link)
sub_soup = BeautifulSoup(sub_res.text, 'html.parser')

In [47]:
sub_lis = sub_soup.select('.basic_list7>li')

In [48]:
name = sub_lis[0].get_text().strip()[6:]

In [49]:
addr = sub_lis[1].get_text().strip()[9:-9]
addr

'경기도 수원시 장안구 정자천로 199'

In [50]:
lines = []
for li in lis:
    link = li.find('a')['href']
    sub_res = requests.get(link)
    sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
    sub_lis = sub_soup.select('.basic_list7>li')
    name = sub_lis[0].get_text().strip()[6:]
    addr = sub_lis[1].get_text().strip()[9:-9]
    lines.append({'name':name,'addr':addr})


IndexError: list index out of range

In [ ]:
# 에러해결
li6=lis[6]
link = li.find('a')['href']
print(link)
sub_res = requests.get(link)
sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
sub_lis = sub_soup.select('.basic_list7>li')
name = sub_lis[0].get_text().strip()[6:]
addr = sub_lis[1].get_text().strip()[9:-9]
print(name,addr)


https://www.ggpolice.go.kr/gunpo/html.do?menu=HC24


IndexError: list index out of range

In [51]:
lines = []
for li in lis:
    link = li.find('a')['href']
    sub_res = requests.get(link)
    sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
    sub_lis = sub_soup.select('.basic_list7>li')
    if len(sub_lis) ==0:
        continue
    name = sub_lis[0].get_text().strip()[6:]
    addr = sub_lis[1].get_text().strip()[9:-9]
    lines.append({'name':name,'addr':addr})


In [52]:
df = pd.DataFrame(lines)

In [62]:
# 중복제거
df.drop_duplicates(subset='name', inplace=True)
df.shape


(28, 2)

In [63]:
df.shape

(28, 2)

In [64]:
import folium

In [66]:
with open('../04.지도시각화/data/kakaoapikey.txt') as f:
    kakao_key = f.read()

In [67]:
base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
header = {'Authorization': f'KakaoAK {kakao_key}'}
lat_list, lng_list = [], []
for i in df.index:
    url = f'{base_url}?query={quote(df.addr[i])}'
    result = requests.get(url, headers=header).json()
    lat_list.append(float(result['documents'][0]['y']))
    lng_list.append(float(result['documents'][0]['x']))
df['위도'] = lat_list
df['경도'] = lng_list
df.head()

,name,addr,위도,경도
0,수원중부경찰서,경기도 수원시 장안구 정자천로 199,37.297932,126.996317
1,수원남부경찰서,경기 수원시 영통구 매봉로 52,37.272104,127.054492
2,수원서부경찰서,경기 수원시 권선구 서부로 1673,37.258740,126.972264
3,안양동안경찰서,경기도 안양시 동안구 동안로159번길 43,37.391209,126.948866
4,안양만안경찰서,경기도 안양시 만안구 냉천로 63,37.387334,126.925974


In [68]:
police = folium.Map(location=[df.위도.mean(), df.경도.mean()], zoom_start=10)

for i in df.index:
    folium.Marker(
        location=[df.위도[i], df.경도[i]],
        popup=folium.Popup(df.addr[i], max_width=200),
        tooltip=df.name[i]
    ).add_to(police)

title_html = '<h3 align="center" style="font-size:20px">경기남부경찰청 산하 경찰서</h3>'
police.get_root().html.add_child(folium.Element(title_html))
police